In [1]:
#include <iostream>
#include <xtensor/xarray.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xview.hpp>

#include <xtensor/xarray.hpp>
#include <xtensor/xfixed.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xtensor.hpp>
#include <xtensor/xadapt.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xaxis_slice_iterator.hpp>
#include <xtensor/xaxis_iterator.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xindex_view.hpp>

#include <xtensor/xchunked_array.hpp>
#include <xtensor/xfixed.hpp>
#include <cstddef>
#include <vector>
#include <xtensor/xarray.hpp>
#include <xtensor/xadapt.hpp>

using namespace xt;

In [2]:
using namespace std;

In [3]:
#include <istream>
#include <fstream>
#include <iostream>

#include <xtensor/xarray.hpp>
#include <xtensor/xcsv.hpp>

#include <iostream>
#include <fstream>
using namespace std;

ifstream myfile;
myfile.open ("lecture_dataset.txt");

auto data = xt::load_csv<double>(myfile);
myfile.close();

data.reshape(data.shape());

In [4]:
xarray<double> X = view(data, all(), range(0, 2));

In [5]:
// auto get_neighbors(int current_index, const xarray<int>& X) {
//     list<int> neighbor_indices = list<int>();
//     for (int neighbor_index : arange(0, n)){
//         if (euclidean_distance(row(X, neighbor_index), row(X, current_index)) <= epsilon){
//             neighbor_indices.push_back((int) neighbor_index);
//             // std::cout << (int) neighbor_index << std::endl;
//         };
//     }
//     return neighbor_indices;
// };

// // extend cluster
// void search(int current_index, int minPts, const xarray<int>& X) {
//     list<int> neighbor_indices = get_neighbors(current_index);
//     if (neighbor_indices.size() >= minPts){
//         state(current_index) = CLUSTERED;
//         cluster(current_index) = cluster_id;
//         for(int neighbor_index=0; neighbor_index<neighbor_indices.size(); neighbor_index++){
//             if (state(neighbor_index)==VISITED || state(neighbor_index)==NOT_VISITED){
//                 state(neighbor_index) = CLUSTERED;
//                 cluster(neighbor_index) = cluster_id;
//                 std::cout << neighbor_index << std::endl;
//                 search(neighbor_index);
//             };
//         };
//     };
// };

In [5]:
#include <functional>

In [44]:
sqrt(sum(pow(row(X, 0) - row(X, 1), 2))(0))

1.7029386

In [5]:
float euclidean_distance(const xarray<double>& a, const xarray<double>& b){ 
    return (float) sqrt(sum(pow((a - b), 2))(0));
}

In [27]:


auto dbscan(const xarray<float>& X, float epsilon, int minPts)
{
    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    
    auto get_neighbors = [&] (int current_index) {
        list<int> neighbor_indices = list<int>();
        for (int neighbor_index : arange(0, n)){
            // std::cout << (float)row(X, current_index)(0) << ',' << (float)row(X, current_index)(1) << ';' << (float)row(X, neighbor_index)(0) << ',' <<(float)row(X, neighbor_index)(1) << std::endl;
            // std::cout << euclidean_distance(row(X, neighbor_index), row(X, current_index)) << std::endl;
            // std::cout << (int) current_index  << (int) neighbor_index << "\n";
            if (euclidean_distance(row(X, neighbor_index), row(X, current_index)) <= (float) epsilon){
                neighbor_indices.push_back((int) neighbor_index);
                std::cout << (int) current_index  << (int) neighbor_index << "\n";
                
                // int x = 0;
                // std::cin >> x;
            };
        }
        return neighbor_indices;
    };
    
    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        list<int> neighbor_indices = get_neighbors(current_index);
        // std::cout << (int) neighbor_indices.size() << std::endl;
        if ((int) neighbor_indices.size() >= minPts){
            state(current_index) = CLUSTERED;
            cluster(current_index) = cluster_id;
            std::cout << current_index << ',' << (int) cluster(current_index) << std::endl;
            for(int neighbor_index=0; neighbor_index<neighbor_indices.size(); neighbor_index++){
                std::cout << neighbor_index << std::endl;
                if (state(neighbor_index) == VISITED || state(neighbor_index) == NOT_VISITED){
                    state(neighbor_index) = CLUSTERED;
                    cluster(neighbor_index) = cluster_id;
                    search(neighbor_index);
                };
            };
        };
    };

    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        // std::cout << (int) idx(0,0) << std::endl;
        search(idx(0,0));
        state(idx(0,0)) = CLUSTERED;
        cluster_id++;
    }
    return cluster;
}

In [28]:
dbscan(X, 2, 4)

4.2,4;4.2,4
0
00
4.2,4;5.9,3.9
1.70294
01
4.2,4;2.8,3.5
1.48661
02
4.2,4;12,1.3
8.25409
03
4.2,4;10,1.3
6.39766
04
4.2,4;1.1,3
3.2573
05
4.2,4;0,2.4
4.49444
06
4.2,4;2.4,2
2.69072
07
4.2,4;11.5,1.8
7.6243
08
4.2,4;11,1
7.43236
09
4.2,4;0.9,0
5.18556
010
4.2,4;1,1.5
4.06079
011
5.9,3.9;4.2,4
1.70294
10
5.9,3.9;5.9,3.9
0
11
5.9,3.9;2.8,3.5
3.1257
12
5.9,3.9;12,1.3
6.63099
13
5.9,3.9;10,1.3
4.85489
14
5.9,3.9;1.1,3
4.88365
15
5.9,3.9;0,2.4
6.08769
16
5.9,3.9;2.4,2
3.98246
17
5.9,3.9;11.5,1.8
5.9808
18
5.9,3.9;11,1
5.86686
19
5.9,3.9;0.9,0
6.34114
110
5.9,3.9;1,1.5
5.45619
111
2.8,3.5;4.2,4
1.48661
20
2.8,3.5;5.9,3.9
3.1257
21
2.8,3.5;2.8,3.5
0
22
2.8,3.5;12,1.3
9.45939
23
2.8,3.5;10,1.3
7.52861
24
2.8,3.5;1.1,3
1.772
25
2.8,3.5;0,2.4
3.00832
26
2.8,3.5;2.4,2
1.55242
27
2.8,3.5;11.5,1.8
8.86454
28
2.8,3.5;11,1
8.57263
29
2.8,3.5;0.9,0
3.98246
210
2.8,3.5;1,1.5
2.69072
211
2,3
0
1
2
3
12,1.3;4.2,4
8.25409
30
12,1.3;5.9,3.9
6.63099
31
12,1.3;2.8,3.5
9.45939
32
12,1.3;12,1.3
0
33
12,1.3;10,1.

-1.
-1.
3.
3.
4.
5.
-1.
7.
8.
9.
-1.


In [22]:
auto euclidean_distance(const xarray<double>& a, const xarray<double>& b){ 
    return sqrt(sum(pow((a - b), 2)))(0);
};

In [23]:
euclidean_distance(row(X, 0), row(X, 1))

1.7029386

In [29]:
xarray<double> cluster = ones<double>({10}) * -1;
int NOT_VISITED = -1;
int VISITED = 0;
(cluster(2) == NOT_VISITED || cluster(2) == VISITED)

true

In [4]:
xarray<double> X = zeros<double>({3});

In [ ]:
#include "random.hpp"
#include "xplot/xfigure.hpp"
#include "xplot/xmarks.hpp"
#include "xplot/xaxes.hpp"

In [5]:
void dodaj_papierosy_do( xarray<double>& X, int i) {
    X(i)=0;
    f(X, 2);
    return;
}

In [ ]:
f(X, 0)

In [45]:
list<int> myList = list<int>();
myList.push_back(4);
myList.size()

1

In [ ]:
4.2	4.0	1
5.9	3.9	0
2.8	3.5	1
12.0	1.3	2
10.0	1.3	2
1.1	3.0	1
0.0	2.4	1
2.4	2.0	1
11.5	1.8	2
11.0	1.0	2
0.9	0.0	1
1.0	1.5	1

In [4]:
// xarray<double> X = zeros<double>({3, 2});

xarray<double> X = {{1, 2},
                    {5, 6}};
X

1.,2.
5.,6.


In [10]:
euclidean_distance(view(X, 0, all()), view(X, 1, all()))

0


In [18]:
sqrt(sum(pow((row(X, 0) - row(X, 1)), 2)))(0)>4

true

In [11]:
sqrt(sum(pow((a - b), 2)))

5.65685


In [3]:
xarray<double> a = row(X, 0);
xarray<double> b = row(X, 1);

In [5]:
euclidean_distance(a,b)

5.65685


In [3]:
auto example_xarray(const xarray<double>& m)
{
    return m + 2;
}

In [4]:
X()

0.,0.
0.,0.
0.,0.


In [6]:
auto a = row(X, 0)
auto b = row(X, 1)

example_xarray(a,b)

input_line_17:2:20: error: expected ';' at end of declaration
 auto a = row(X, 0)
                   ^
                   ;
input_line_17:3:19: error: expected ';' at end of declaration
auto b = row(X, 1)
                  ^
                  ;


Interpreter Error: 

In [3]:
row(X, 0)

0.
0.


In [ ]:
def euclidean_distance(a, b):
    distance_measure = 0
    for i in range(len(a)):
         distance_measure += pow((a[i] - b[i]), 2)
    return np.sqrt(distance_measure)

In [32]:
xarray<double> X = zeros<double>({3});

In [5]:
for (int x : arange(0, n))
{
    std::cout << x << std::endl;
}

input_line_18:2:25: error: use of undeclared identifier 'n'
 for (int x : arange(0, n))
                        ^


Interpreter Error: 